# Kinematic Wave Parameter Estimation

## Requirements

This module uses information from the USGS stations over the CONUS to generate a statistical model for estimating the kinematic wave alpha and beta parameters for a given grid. The already trained model is provided here.

This model requires basin averaged estimates of the following variables:
* Mean annual temperature (degrees C)
* Mean annual precipitation (mm)
* Impervious area (%)
* Clay (%)
* Sand (%)
* Silt (%)
* Relief ratio
* Course Fragments (%)
* Depth to bedrock (mm)
* Bulk Density
* Population

In [15]:
from ef5_kw_estimate import ReadGrid, WriteGrid
import numpy as np
import pickle

## Read in the input grids

In [9]:
# keepInfo=True keeps the projection and spatial reference information for this grid...
# to use when writing the output grids
# We need log basin area, so compute that here
basinArea = ReadGrid("input_grids/basin.area.tif", keepInfo=True)
basinArea = np.log10(basinArea)

temp = ReadGrid("input_grids/temp.avg.tif")
precip = ReadGrid("input_grids/precip.avg.tif")
imperv = ReadGrid("input_grids/imperv.avg.tif")
clay = ReadGrid("input_grids/clay_pct.avg.tif")
sand = ReadGrid("input_grids/sand_pct.avg.tif")
silt = ReadGrid("input_grids/silt_pct.avg.tif")
rr = ReadGrid("input_grids/relief.ratio.avg.tif")
frags = ReadGrid("input_grids/course_fragments.avg.tif")
bedrock = ReadGrid("input_grids/abs_depth_bedrock.avg.tif")
density = ReadGrid("input_grids/bulk_density.avg.tif")
pop = ReadGrid("input_grids/population.avg.tif")

## Load the pickled models so we can run them on our data

In [16]:
scaler = pickle.load(open("scaler.p", "rb"), encoding='latin1')
alphamod = pickle.load(open("alpha_model.p", "rb"), encoding='latin1')
betamod = pickle.load(open("beta_model.p", "rb"), encoding='latin1')

/usr/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator StandardScaler from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVR from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


## Run the models to estimate alpha & beta

In [17]:
print("Transforming to scaled parameter space")
pred_real = scaler.transform(np.column_stack((basinArea, temp, precip, clay, sand, silt, rr, frags, bedrock, imperv, density, pop)))

print("Computing alpha")
alpha = alphamod.predict(pred_real)
alpha = np.power(10.0, alpha)

print("Computing beta")
beta = betamod.predict(pred_real)

# Since the model extrapolates, we do a trick here to bound the minimum beta to 0.
bad = np.where(beta < 0.0)
beta[bad] = np.exp(beta[bad] * 10.0)

Transforming to scaled parameter space
Computing alpha
Computing beta


## Write the new parameter grids to disk

In [18]:
WriteGrid("output_grids/kw_alpha.tif", alpha)
WriteGrid("output_grids/kw_beta.tif", beta)